# setup



In [ ]:

#!pip install -q --upgrade ipython
#!pip install -q --upgrade ipykernel
#connect to gdrive
from google.colab import drive
drive.mount('/content/gdrive')
%cd gdrive/MyDrive
#%cd ../..
!pwd

In [ ]:
%cd competitions/11785-spring2021-hw2p2s1-face-classification

/content/gdrive/MyDrive/competitions/11785-spring2021-hw2p2s1-face-classification


In [ ]:
!pip install kaggle
!mkdir .kaggle
import json
token = {"username":"yuzhongfinally","key":"da5ea7094c4adb8f29251a2e3b240da7"}
with open('/content/gdrive/MyDrive/.kaggle/kaggle.json', 'w') as file:
    json.dump(token, file)
!chmod 600 /content/gdrive/MyDrive/.kaggle/kaggle.json
!cp /content/gdrive/MyDrive/.kaggle/kaggle.json /root/.kaggle/
!kaggle config set -n path -v /content/gdrive/MyDrive/

In [ ]:
!pip install --upgrade --force-reinstall --no-deps kaggle
!kaggle competitions download -c 11785-spring2021-hw2p2s1-face-classification
!pwd

In [ ]:
%cd competitions/11785-spring2021-hw2p2s1-face-classification
!pwd

In [ ]:
#%cd competitions
!pwd
!unzip 11785-spring2021-hw2p2s1-face-classification.zip

In [ ]:

import numpy as np
import torch
import torch.nn as nn
import torchvision

In [ ]:
%cd 11785-spring2021-hw2p2s1-face-classification


[Errno 2] No such file or directory: '11785-spring2021-hw2p2s1-face-classification'
/content/gdrive/MyDrive/competitions/11785-spring2021-hw2p2s1-face-classification


In [ ]:
import PIL
transforms = torchvision.transforms.Compose([

    torchvision.transforms.ColorJitter(hue=.05, saturation=.05),
    torchvision.transforms.RandomHorizontalFlip(),
    #torchvision.transforms.RandomRotation(20, resample=PIL.Image.BILINEAR),
    torchvision.transforms.ToTensor()
])
train_dataset = torchvision.datasets.ImageFolder(root='train_data', 
                                                 transform = transforms)
train_dataloader = torch.utils.data.DataLoader(train_dataset, batch_size=128, 
                                               shuffle=True, num_workers=1)

In [ ]:
val_dataset = torchvision.datasets.ImageFolder(root='val_data', 
                                                 transform= torchvision.transforms.ToTensor())
val_dataloader = torch.utils.data.DataLoader(val_dataset, batch_size=128, 
                                               shuffle=False, num_workers=1)

# training


In [ ]:
class SimpleResidualBlock(torch.nn.Module):
    def __init__(self, in_channels, out_channels ,stride=1, downsample = False):
        super(SimpleResidualBlock, self).__init__()
        self.downsample = downsample
        self.expansion = 1
        self.conv1 = nn.Conv2d(in_channels, out_channels,kernel_size = 1,stride = 1, padding = 0, bias = False)
        self.bn1 = nn.BatchNorm2d(out_channels)
        self.relu = nn.ReLU()
        self.dropout = nn.Dropout(p= 0.1)
        self.conv2 = nn.Conv2d(out_channels,out_channels,kernel_size = 3,stride = stride, padding = 1, bias=False)
        self.bn2 = nn.BatchNorm2d(out_channels)
        
        self.shortcut = nn.Sequential()
        if downsample == False:
            self.shortcut = nn.Identity()
        else:
     
            self.shortcut = nn.Conv2d(in_channels,out_channels, kernel_size=1, stride=stride, bias = False)
            self.bn3 = nn.BatchNorm2d(out_channels)
       # self.relu = nn.ReLU(inplace = True)
    
    def forward(self, x):
      
        out = self.conv1(x)
        out = self.bn1(out)
        out = self.relu(out)
        out = self.dropout(out)
        out = self.conv2(out)
        out = self.bn2(out)

        if self.downsample == True:
          shortcut = self.bn3(self.shortcut(x))
        else:
          shortcut = self.shortcut(x)
        #print(shortcut.shape)
       
        out = self.relu(out + shortcut)
        #out = out + shortcut
        
        return out

In [ ]:

class ResNet(torch.nn.Module):
  def __init__(self, block, layers,num_classes, feat_dim):
    super(ResNet, self).__init__()
    self.in_channels = 32
    self.conv1 = nn.Conv2d(3,32, kernel_size = 3, stride = 1, padding = 1, bias = False )
    self.bn1 = nn.BatchNorm2d(32)
    
    self.relu = nn.ReLU()
    self.maxpool = nn.MaxPool2d(kernel_size = 2, stride = 2, padding = 1)

 
    #for resnet layers
    self.layer1 = self.sublayer(block, layers[0], channel_out = 32, stride = 1)
    self.layer2 = self.sublayer(block, layers[1], channel_out = 48, stride = 1)
    self.layer3 = self.sublayer(block, layers[2], channel_out = 96, stride = 1)
    self.layer4 = self.sublayer(block, layers[3], channel_out = 192, stride = 2) 
    self.bn2 = nn.BatchNorm2d(192*block(2000,4000).expansion)
   
    self.avgpool =  nn.AdaptiveAvgPool2d((1, 1))
    self.flatten = nn.Flatten()
    
    self.linear = nn.Linear(self.in_channels, feat_dim)
    
    self.linear_output = nn.Linear(192*block(2000, 4000).expansion, num_classes)
    
    
  
  def sublayer(self, block , num_block, channel_out, stride ):
   
    expansion = block(64,64,1).expansion
    layers = []
    for s in range(num_block):
      if s == 0:
        if expansion*channel_out != self.in_channels: # test if the in channel and out channel are the same
          layers.append(block(self.in_channels, channel_out, downsample = True))
        else:
          layers.append(block(channel_out, channel_out))
        self.in_channels = channel_out
      else:
        layers.append(block(self.in_channels*expansion, channel_out, stride, downsample = True))
    self.in_channels = channel_out * expansion
    return nn.Sequential(*layers)

  def forward(self, x, return_embedding = False):

    x = self.conv1(x)
    x = self.bn1(x)
    x = self.relu(x)
    x = self.maxpool(x)

    x = self.layer1(x)  
    x = self.layer2(x)
    x = self.layer3(x)
    x = self.layer4(x)
    x = self.bn2(x)
    x = self.avgpool(x)
   
    x = self.flatten(x)
    
    embedding_out = self.relu(self.linear(x))

    output = self.linear_output(x)
    if return_embedding:
      return embedding_out, output
    else:
      return output





In [ ]:

def ResNet34(num_classes, feat_dim):
  return ResNet(SimpleResidualBlock, [3,4,6,3],  num_classes, feat_dim)


In [ ]:
def training(epoch, network):
  for epoch in range(numEpochs):
      
      network.train()
      avg_loss = 0.0
      num_correct = 0

      for batch_num, (x, y) in enumerate(train_dataloader):
          optimizer.zero_grad()
          
          x, y = x.to(device), y.to(device)

          outputs = network(x)

          loss = criterion(outputs, y.long())
          loss.backward()
          optimizer.step()
          
          avg_loss += loss.item()
          if batch_num % 99 == 1:
              print('loss', avg_loss/100)
              avg_loss = 0.0
       
          #print(torch.argmax(outputs, axis = 1)[0],y[0])
          num_correct += (torch.argmax(outputs, axis=1) == y).sum().item()
      
      
      print('Epoch: {}, Training Accuracy: {:.2f}'.format(epoch, num_correct / len(train_dataset)))
      print('train loss',avg_loss)
      
      # Validate
      network.eval()
      num_correct = 0
      avg_loss = 0.0
      with torch.no_grad():
        for batch_num, (x, y) in enumerate(val_dataloader):
            x, y = x.to(device), y.to(device)
            outputs = network(x)
            loss = criterion(outputs, y.long())
            avg_loss += loss.item()/len(val_dataloader)
            
            if batch_num % 99 == 1:
              print('loss', avg_loss)
              avg_loss = 0.0
            
            #print(torch.argmax(outputs, axis = 1)[0],y[0])
            num_correct += (torch.argmax(outputs, axis=1) == y).sum().item()

        print('val loss', avg_loss)
        print('Epoch: {}, Validation Accuracy: {:.2f}'.format(epoch, num_correct / len(val_dataset)))
        acc = num_correct / len(val_dataset)
        scheduler.step(acc)
        if num_correct/len(val_dataset) >= 0.7:
          torch.save(network,'resnet_0.7x2')


# tuning hyperparameters


In [ ]:
#different feature dimensions
num_classes = len(train_dataset.classes)
feat_dim_list = [1200]

for feat_dim in feat_dim_list:
  print(feat_dim)
  
  learningRate = 0.1
  
  numEpochs = 25

  in_features = 3 # RGB channels

  weightDecay = 5e-5 #5e-5

  device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
  
  network = ResNet34(num_classes, feat_dim)
  network = network.to(device)
  
  criterion = nn.CrossEntropyLoss()
  optimizer = torch.optim.SGD(network.parameters(), lr=learningRate, weight_decay=weightDecay, momentum=0.9)
  
  scheduler = torch.optim.lr_scheduler.StepLR(optimizer, step_size=10, gamma=0.2)
  training(numEpochs, network)
  
  



1200
loss 0.1660374927520752
loss 8.218052349090577
loss 8.170220584869385
loss 8.123944873809814
loss 8.066201248168944
loss 8.002848262786864
loss 7.963303184509277
loss 7.895539546012879
loss 7.834396810531616
loss 7.787184309959412
loss 7.702411684989929
loss 7.639333424568176
loss 7.593445725440979
loss 7.536490778923035
loss 7.492283821105957
loss 7.437731657028198
loss 7.372276301383972
loss 7.325784769058227
loss 7.281984672546387
loss 7.201012101173401
loss 7.173785462379455
loss 7.105573892593384
loss 7.0541437196731565
loss 7.024293284416199
loss 6.926671710014343
loss 6.881153025627136
loss 6.810311598777771
loss 6.756075692176819
loss 6.690614399909973
loss 6.635176043510437
loss 6.5747545719146725
Epoch: 0, Training Accuracy: 0.01
train loss 13.199355602264404
loss 0.2178756168910435
val loss 6.673875452980164
Epoch: 0, Validation Accuracy: 0.01
loss 0.13135613918304442
loss 6.463390817642212
loss 6.4366775369644165
loss 6.389204244613648
loss 6.329658374786377
loss 6.262

KeyboardInterrupt: ignored

In [ ]:
with torch.no_grad():
        for batch_num, (x, y) in enumerate(val_dataloader):
            x, y = x.to(device), y.to(device)
            outputs = network_val(x)
            loss = criterion(outputs, y.long())
            avg_loss += loss.item()/len(val_dataloader)
            
            if batch_num % 99 == 1:
              print('loss', avg_loss)
              avg_loss = 0.0
            
            #print(torch.argmax(outputs, axis = 1)[0],y[0])
            num_correct += (torch.argmax(outputs, axis=1) == y).sum().item()

        print('val loss', avg_loss)
        print('Validation Accuracy: {:.2f}'.format(num_correct / len(val_dataset)))

loss 6.145896334496756
val loss 1.0863212952538142
Validation Accuracy: 0.80


# testing

In [ ]:
#read the test image

from torch.utils.data import DataLoader,Dataset
from PIL import Image
from os import listdir
from os.path import isfile, join
class ImageDataset(Dataset):
    def __init__(self, file_list):
        self.file_list = file_list
        self.imgfiles = [f for f in listdir(self.file_list) if isfile(join(self.file_list, f))]
        

    def __len__(self):
        return len(self.imgfiles)

    def __getitem__(self, index):
        fname = self.file_list + '/'+ self.imgfiles[index]
        
        img = Image.open(fname)
        img = torchvision.transforms.ToTensor()(img)

        return img,self.imgfiles[index]

In [ ]:
testset = ImageDataset('test_data')
test_loader = torch.utils.data.DataLoader(testset, batch_size=128, 
                                               shuffle=False, num_workers=1)

In [ ]:
network.eval()
preds_total = []
names = []
with torch.no_grad():
  for X_test, name in test_loader:
   # print(name)
    names.append(name)
    X_test = X_test.to(device='cuda')
    y_hat = network(X_test)
    
    y_hat = y_hat.cpu()
    y_hat = y_hat.detach().numpy()
    
    preds = np.argmax(y_hat, axis = 1)
   
    preds = preds.reshape(-1,1)
    preds_total.append(preds)

In [ ]:
dictclass = train_dataset.class_to_idx
dictclass.keys()

In [ ]:
dictclass.values()

In [ ]:
finalnames = []
for k in names:
  #print(k)

  for n in range(0,len(k)):
    #print(k[n])

    finalnames.append(k[n])

In [ ]:

preds_final = []
for i in preds_total:
  for t in i:
    
    add = [key for key, val in dictclass.items() if val == t]
    #print(add[0], t)
    preds_final.append(int(add[0]))
len(preds_final)

8000

In [ ]:
import pandas as pd
df = pd.DataFrame({'id':finalnames, "label":preds_final})
df.to_csv(r"submissionp1.csv", index=False)

In [ ]:
!kaggle competitions submit -c 11785-spring2021-hw2p2s1-face-classification -f submissionp1.csv -m "Message"

100% 106k/106k [00:08<00:00, 12.5kB/s]
Successfully submitted to 11785-Spring2021-HW2P2S1-Face-Classification

In [ ]:
del train_dataset

# task2 pre-processing


In [ ]:
import os
import pandas as pd
import torchvision
import torchvision.datasets as dset
import torchvision.transforms as transforms
from torch.utils.data import DataLoader,Dataset
from PIL import Image
import numpy as np
import torch.nn as nn
import torch

In [ ]:
%cd competitions/11785-spring2021-hw2p2s2-face-verification


/content/gdrive/MyDrive/competitions/11785-spring2021-hw2p2s2-face-verification


In [ ]:
!unzip 11785-spring2021-hw2p2s2-face-verification.zip

Streaming output truncated to the last 5000 lines.
  inflating: verification_data/00064100.jpg  
  inflating: verification_data/00064101.jpg  
  inflating: verification_data/00064102.jpg  
  inflating: verification_data/00069096.jpg  


In [ ]:
%cd 11785-spring2021-hw2p2s2-face-verification

/content/gdrive/MyDrive/competitions/11785-spring2021-hw2p2s2-face-verification


In [ ]:
from PIL import Image
import PIL.ImageOps  
import pandas as pd
dir = 'verification_pairs_val.txt'
compare = pd.read_csv(dir, header = None, delimiter = "\t")
text = []
for c in compare[0]:
  i1, i2, l = c.split(" ")
  text.append([i1,i2,l])

  
class TrainDataset(Dataset):
    
    def __init__(self,imageFolderDataset,text,transform=None,should_invert=False, test = False):
        
        
        self.imageFolderDataset = imageFolderDataset    
        self.transform = transform
        self.should_invert = should_invert
        self.image = [[t[0],t[1]] for t in text]
        self.label = [t[2] for t in text]
        self.test = test


    def __getitem__(self,index):

        dirname0 =  text[index][0]
        dirname1 =  text[index][1]
        label = text[index][2]
       
        img0 = Image.open(dirname0)
        img1 = Image.open(dirname1)
        
        
        img0 = torchvision.transforms.ColorJitter(hue=.05, saturation=.05)(img0) 
        img0 = torchvision.transforms.RandomHorizontalFlip()(img0)
        img0 = transforms.ToTensor()(img0)
        img1 = transforms.ToTensor()(img1)
        img1 = torchvision.transforms.ColorJitter(hue=.05, saturation=.05)(img1) 
        img1 = torchvision.transforms.RandomHorizontalFlip()(img1)
        
      
        if self.should_invert:
            img0 = PIL.ImageOps.invert(img0)
            img1 = PIL.ImageOps.invert(img1)

        if self.transform is not None:
            img0 = self.transform(img0)
            img1 = self.transform(img1)

    

        if self.test == False:
          l = torch.from_numpy(np.array(label, dtype = np.float32))
          return img0, img1, l
        else:
          return img0, img1 ,0
    
    def __len__(self):
        return len(self.image)

In [ ]:

train_dataset = TrainDataset(dir, text)
train_dataloader = DataLoader(train_dataset,
                        shuffle=True,
                        num_workers=0,
                        batch_size=1)

# task2 Cosine Similarity

In [ ]:
compute_sim = nn.CosineSimilarity(dim=0)
#print("CS b/n two images of class 0: {:.4f}".format(compute_sim(feats_0, feats_1)))
def testVerify(model, train_dataloader):
    similarity = np.array([])
    true = np.array([])
    
    model.eval()
    with torch.no_grad():
        for batch_idx, (img0, img1, targets) in enumerate(train_dataloader):
            img0, img1, targets = img0.cuda(), img1.cuda(), targets.cuda()
          
          
            feats_0 = model(img0, return_embedding=True)[0].squeeze(0)
            feats_1 = model(img1, return_embedding=True)[0].squeeze(0)
      
            sim = compute_sim(feats_0,feats_1)
                     
            similarity = np.concatenate((similarity, sim.cpu().numpy().reshape(-1)))

            true = np.concatenate((true, targets.cpu().numpy().reshape(-1)))
            
            del img0
            del img1
            del targets
    return similarity, true

In [ ]:
%cd .. 
%cd 11785-spring2021-hw2p2s2-face-verification
similarity, true = testVerify(network, train_dataloader)

/content/gdrive/MyDrive/competitions
/content/gdrive/MyDrive/competitions/11785-spring2021-hw2p2s2-face-verification


In [ ]:
from sklearn.metrics import roc_auc_score
auc = roc_auc_score(true, similarity)
print(auc)

0.8960510933270018


# task2 test

In [ ]:
%cd .. 
%cd 11785-spring2021-hw2p2s1-face-classification
network = torch.load('resnet_0.7x2')

/content/gdrive/MyDrive/competitions
/content/gdrive/MyDrive/competitions/11785-spring2021-hw2p2s1-face-classification


In [ ]:
dir = 'verification_pairs_test.txt'
text = []
compare = pd.read_csv(dir, header = None, delimiter = "\t")
for c in compare[0]:
  i1, i2 = c.split(" ")
  text.append([i1,i2,0])

test_dataset = TrainDataset(dir, text, test = True)
test_dataloader = DataLoader(test_dataset,
                        shuffle=False,
                        num_workers=0,
                        batch_size=1)

In [ ]:

def testVerify_final(model, train_dataloader):
    similarity = np.array([])
    true = np.array([])
    
    model.eval()
    with torch.no_grad():
        for batch_idx, (img0, img1, targets) in enumerate(train_dataloader):
            img0, img1, targets = img0.cuda(), img1.cuda(), targets.cuda()
            # find cos similarity between embeddings
            #print(img0.shape)

          
            feats_0 = model(img0, return_embedding=True)[0].squeeze(0)
            feats_1 = model(img1, return_embedding=True)[0].squeeze(0)
      
            sim = compute_sim(feats_0,feats_1)
           # print('sim', sim)

            similarity = np.concatenate((similarity, sim.cpu().numpy().reshape(-1)))
    
            
            del img0
            del img1
            del targets
    return similarity

In [ ]:
similarity = testVerify_final(network, test_dataloader)

In [ ]:
import pandas as pd
df = pd.DataFrame({'Id':compare[0], "Category":similarity})
df.to_csv(r"submission4.csv", index=False)

In [ ]:
!kaggle competitions submit -c 11785-spring2021-hw2p2s2-face-verification -f submission4.csv -m "Message"

100% 4.02M/4.02M [00:03<00:00, 1.09MB/s]
Successfully submitted to 11785-Spring2021-HW2P2S2-Face-Verification